In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
unfilteredBusinessData=pd.read_json("data/yelp_academic_dataset_business.json", lines=True)

In [3]:
# unfilteredUsers=pd.DataFrame()

# userReviewReader=pd.read_json("data/yelp_academic_dataset_user.json", lines=True, chunksize=50000)
# for chunk in userReviewReader:
#     unfilteredUsers = pd.concat([unfilteredUsers, chunk])

In [3]:
# this takes a long time time to run due to just how much data there is to run through, 
# but it is necessary given the relative lack of memory my computer has.

unfilteredReviews = pd.DataFrame()
reviewReader=pd.read_json("data/yelp_academic_dataset_review.json", lines=True, chunksize=10000)
for chunk in reviewReader:
    unfilteredReviews = pd.concat([unfilteredReviews, chunk])

In [4]:
# Select all businesses with "Restaurants" category
restaurants_df = unfilteredBusinessData[(unfilteredBusinessData["categories"].str.contains("restaurants", case=False, na=False))]

# Select all restaurants with > 30 reviews
reviewed_restaurants =  restaurants_df[(restaurants_df["review_count"] >= 30)]

In [5]:
# unfilteredReviews.head()

In [6]:
# select the relevant columns for grouping
businessStars=unfilteredReviews[["business_id", "stars"]]
# businessStars.head()
groupedReviews=businessStars.groupby("business_id").mean()

In [10]:
groupedReviewsDF=groupedReviews.reset_index()
groupedReviewsDF=groupedReviewsDF.rename(columns={"business_id": "business_id", "stars" : "average_stars"})
groupedReviewsDF.head()
# join to add average stars to each business.
joiningDF=pd.merge(reviewed_restaurants, groupedReviewsDF, how="inner", on="business_id")
joiningDF.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,average_stars
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",4.057471
1,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.916116,-82.760461,4.5,100,0,"{'OutdoorSeating': 'False', 'RestaurantsGoodFo...","Food, Delis, Italian, Bakeries, Restaurants","{'Monday': '10:0-18:0', 'Tuesday': '10:0-20:0'...",4.386792
2,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,Philadelphia,PA,19106,39.953949,-75.143226,4.0,245,1,"{'RestaurantsReservations': 'True', 'Restauran...","Sushi Bars, Restaurants, Japanese","{'Tuesday': '13:30-22:0', 'Wednesday': '13:30-...",4.200000
3,ROeacJQwBeh05Rqg7F6TCg,BAP,1224 South St,Philadelphia,PA,19147,39.943223,-75.162568,4.5,205,1,"{'NoiseLevel': 'u'quiet'', 'GoodForMeal': '{'d...","Korean, Restaurants","{'Monday': '11:30-20:30', 'Tuesday': '11:30-20...",4.317308
4,WKMJwqnfZKsAae75RMP6jA,Roast Coffeehouse and Wine Bar,10359 104 Street NW,Edmonton,AB,T5J 1B9,53.546045,-113.499169,4.0,40,0,"{'OutdoorSeating': 'False', 'Caters': 'True', ...","Coffee & Tea, Food, Cafes, Bars, Wine Bars, Re...","{'Monday': '8:0-18:0', 'Tuesday': '8:0-18:0', ...",3.825000
